In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
from fifa_storage import fifa_pw, fifa_username

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("players_20.csv")
df

In [ ]:
df.describe()

In [ ]:
df.columns.values

In [ ]:
df1 = df[['sofifa_id','player_url','short_name', 'age','nationality', 'club', 'overall', 'wage_eur', 'player_positions', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']].copy()

In [ ]:
# List of countries for each continent
continents = {
    'Africa' : ['Algeria','Angola','Benin','Botswana','Burkina','Burundi','Cameroon','Cape Verde','Central African Republic','Chad','Comoros','Congo','DR Congo','Djibouti','Egypt','Equatorial Guinea','Eritrea','Ethiopia','Gabon','Gambia','Ghana','Guinea','Guinea Bissau','Ivory Coast','Kenya','Lesotho','Liberia','Libya','Madagascar','Malawi','Mali','Mauritania','Mauritius','Morocco','Mozambique','Namibia','Niger','Nigeria','Rwanda','Sao Tome and Principe','Senegal','Seychelles','Sierra Leone','Somalia','South Africa','South Sudan','Sudan','Swaziland','Tanzania','Togo','Tunisia','Uganda','Zambia','Zimbabwe','Burkina Faso'],
    'Antarctica' : ['Fiji','Kiribati','Marshall Islands','Micronesia','Nauru','New Zealand','Palau','Papua New Guinea','Samoa','Solomon Islands','Tonga','Tuvalu','Vanuatu'],
    'Asia' : ['Afghanistan','Bahrain','Bangladesh','Bhutan','Brunei','Burma (Myanmar)','Cambodia','China','China PR','East Timor','India','Indonesia','Iran','Iraq','Israel','Japan','Jordan','Kazakhstan','North Korea','South Korea','Korea Republic','Korea DPR','Kuwait','Kyrgyzstan','Laos','Lebanon','Malaysia','Maldives','Mongolia','Nepal','Oman','Pakistan','Palestine','Philippines','Qatar','Russian Federation','Saudi Arabia','Singapore','Sri Lanka','Syria','Tajikistan','Thailand','Turkey','Turkmenistan','United Arab Emirates','Uzbekistan','Vietnam','Yemen','Russia'],
    'Australia Oceania' : ['Australia','New Caledonia'],
    'Europe' : ['Albania','Andorra','Armenia','Austria','Azerbaijan','Belarus','Belgium','Bosnia Herzegovina','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','France','FYR Macedonia','Georgia','Germany','Greece','Hungary','Iceland','Ireland','Italy','Kosovo','Latvia','Liechtenstein','Lithuania','Luxembourg','Macedonia','Malta','Moldova','Monaco','Montenegro','Netherlands','Northern Ireland','Norway','Poland','Portugal','Romania','San Marino','Scotland','Serbia','Slovakia','Slovenia','Spain','Sweden','Switzerland','Ukraine','England','Vatican City','Republic of Ireland','Wales'],
    'North America' : ['Antigua and Barbuda','Bahamas','Barbados','Belize','Canada','Costa Rica','Cuba','Dominica','Dominican Republic','El Salvador','Grenada','Guatemala','Haiti','Honduras','Jamaica','Mexico','Nicaragua','Panama','Saint Kitts and Nevis','Saint Lucia','Saint Vincent and the Grenadines','Trinidad and Tobago','United States'],
    'South America' : ['Argentina','Bolivia','Brazil','Chile','Colombia','Curacao','Ecuador','Guyana','Paraguay','Peru','Suriname','Trinidad & Tobago','Uruguay','Venezuela']
}

In [ ]:
# Function matching continent to countries
def find_continent(x, continents_list):
    # Iteration over 
    for key in continents_list:
        if x in continents_list[key]:
            return key
    return np.NaN

df1['continent'] = df1['nationality'].apply(lambda x: find_continent(x, continents))

In [ ]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json
import IPython.display

In [ ]:
clean = df1.dropna()

In [ ]:
clean.head(50)

In [ ]:
Africa = clean[clean["continent"]=='Africa']
Antarctica = clean[clean["continent"]=='Antarctica']
Asia = clean[clean["continent"]=='Asia']
Australia_Oceania =  clean[clean["continent"]=='Australia_Oceania']
Europe = clean[clean["continent"]=='Europe']
North_america = clean[clean["continent"]=='North_america']
South_america = clean[clean["continent"]=='South_america']

data = {}
data["name"] = "DISTRIBUTION OF TOP 1000 PLAERS DUE TO NATIONALITY"
data["children"] = []
# Split dataset into Continents: thank you Dom and TA's 

for continent in clean['continent'].unique():
    
    continent_set = clean[clean["continent"]==continent]
    continent_dict = {}
    continent_dict["name"] = continent
    continent_dict["children"] = []
    data["children"].append(continent_dict)
    
    for country in continent_set['nationality'].unique():
        
        countries_set = continent_set[continent_set['nationality']==country][['short_name', 'overall']]
        country_dict = {}
        country_dict["name"] = country
        country_dict["children"] = []
        continent_dict['children'].append(country_dict)

        
        for player in countries_set.values:
            
            player_dict = {}
            player_dict['name'] = player[0]
            player_dict['size'] = player[1]
            country_dict["children"].append(player_dict)
            
        

In [ ]:
print(data)

In [ ]:
# North_america_dict = {}
# North_america_dict['name'] = 'North_america'
# North_america_dict['children'] = []
# for country in North_america['nationality'].unique():
#     list_of_countries = North_america[North_america['nationality']==country][['short_name', 'overall']].rename(columns={'short_name': 'name', 'overall': 'size'})
#     tmp_dict = {}
#     tmp_dict["name"] = country
#     tmp_dict["children"] = []
#     for row in list_of_countries.values:
#         player_tmp = {}
#         player_tmp['name'] = row[0]
#         player_tmp['size'] = row[1]
#         tmp_dict["children"].append(player_tmp)
#     North_america_dict['children'].append(tmp_dict)

In [ ]:
# print(North_america_dict)

In [ ]:
clean.to_csv('clean.csv')

In [ ]:
clean

In [ ]:
clean.short_name.count()

In [ ]:
len(clean.short_name.value_counts())

In [ ]:
short_list = clean.groupby('short_name').mean().sort_values('wage_eur',ascending=False)[:200]

In [ ]:
short_list

In [ ]:
short_list.to_csv('short_list.csv')

# Load

In [ ]:
# Connect to local Database

rds_connection_string = f"{fifa_username}:{fifa_pw}@localhost:5432/fifa_df1"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# pulled the table names from our database

engine.table_names()

In [ ]:
#pushed the reset_vaccine dataframe to our table: COUNTRY_VACCINE

clean.to_sql(name='fifa_df1', con=engine, if_exists='append', index=False)

In [ ]:
#read the table: COUNTRY_VACCINE from our database

pd.read_sql_query('select * from fifa_df1', con=engine)